In [2]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
import os
from sklearn.linear_model import LogisticRegression

In [3]:
def quality_metrics(y_true, y_pred, THRESHOLD = 0.5):
  ##### METRICS #####
  # Area under ROC curve
  try:
    AUC = metrics.roc_auc_score(y_true,y_pred)
  except Exception as e:
    print('All values predicited one class!')
    AUC = 0

  # Discrete metrics
  y_pred = y_pred >= THRESHOLD

  TP = np.logical_and(y_pred,y_true).sum()
  TN = np.logical_and(np.logical_not(y_pred), np.logical_not(y_true)).sum()
  FP = np.logical_and(y_pred, np.logical_not(y_true)).sum()
  FN = np.logical_and(np.logical_not(y_pred), y_true).sum()

  # According to definition:
  accuracy = (TP+TN)/len(y_pred)
  sensitivity = TP/(TP+FN)
  specificity = TN/(TN+FP)
  #accuracy = (sensitivity+specificity)/2

  return AUC, accuracy, sensitivity, specificity


In [4]:
def extrapolate_kmeans (img):
    # For some reason (methinks jpg compression) the values are not just 3 but an interpolation. Back to three values:
    # hi_th = np.unique(img)[1:][np.diff(np.unique(img)) != 1]
    unique, counts = np.unique(img, return_counts = True)
    hi_th = unique[np.argpartition(counts, -3)[-3:]]
    img[img <= hi_th[2]] = 0
    #img[np.logical_and(img > hi_th[0], img < hi_th[1])] = hi_th[0] 
    img[np.logical_and(img > hi_th[2], img <= hi_th[1])] = 1
    #img[img >= hi_th[1]] = hi_th[1]
    img[img > hi_th[1]] = 2
    return img

In [3]:
path = "/home/maiki/Downloads/pi_mask_3/h10.jpg"
img = cv2.resize(cv2.imread(path, opt), (256,256))
img_fix = extrapolate_kmeans(img)

#unique[np.argmax[counts]]
#print(np.unique(img)[1:][np.diff(np.unique(img)) != 1])

NameError: name 'opt' is not defined

In [60]:
#imshow(img_fix*127)

np.unique(img_fix*127, return_counts = True)

(array([  0, 127, 254], dtype=uint8), array([26182, 21577, 17777]))

In [6]:
bf_arr = []
pi_m1_arr = []
pi_m2_arr = []

opt = cv2.IMREAD_GRAYSCALE

path = "/home/maiki/Downloads/ht29/"
for img in os.listdir(path+"imgs"):
    print(img+"...")
    bf = cv2.resize(cv2.imread(path+"/imgs/"+img, opt), (256,256))
    bf = np.float32((255-bf.reshape((-1,1))))
    bf = bf/max(bf)
    bf_arr.append(bf)

for img in os.listdir(path+"pi_mask_3"):
    print(img+"...")
    pi_m = cv2.resize(cv2.imread(path+"/pi_mask_3/"+img, opt), (256,256))
    pi_m = extrapolate_kmeans(pi_m)
    pi_m1 = (pi_m == 1).reshape((-1,1))
    pi_m2 = (pi_m == 2).reshape((-1,1))
    
    pi_m1_arr.append(pi_m1)
    pi_m2_arr.append(pi_m2)

c5.jpg...
g8.jpg...
e4.jpg...
b1.jpg...
d1.jpg...
d6.jpg...
g2.jpg...
e9.jpg...
a9.jpg...
h1.jpg...
b5.jpg...
h10.jpg...
g4.jpg...
e1.jpg...
b6.jpg...
h5.jpg...
h2.jpg...
d5.jpg...
e10.jpg...
b2.jpg...
e6.jpg...
c4.jpg...
g9.jpg...
h3.jpg...
a7.jpg...
d9.jpg...
e7.jpg...
h6.jpg...
a6.jpg...
b7.jpg...
f1.jpg...
a5.jpg...
f9.jpg...
g3.jpg...
g5.jpg...
d7.jpg...
h9.jpg...
h7.jpg...
f10.jpg...
c6.jpg...
e2.jpg...
c1.jpg...
e5.jpg...
f8.jpg...
a8.jpg...
d3.jpg...
d4.jpg...
h4.jpg...
g1.jpg...
b8.jpg...
e8.jpg...
g7.jpg...
g6.jpg...
f3.jpg...
a1.jpg...
f4.jpg...
e3.jpg...
c2.jpg...
f7.jpg...
f5.jpg...
f6.jpg...
d2.jpg...
b4.jpg...
a4.jpg...
a2.jpg...
b9.jpg...
f2.jpg...
c5.jpg...
g8.jpg...
e4.jpg...
b1.jpg...
d1.jpg...
d6.jpg...
g2.jpg...
e9.jpg...
a9.jpg...
h1.jpg...
b5.jpg...
h10.jpg...
g4.jpg...
e1.jpg...
b6.jpg...
h5.jpg...
h2.jpg...
d5.jpg...
e10.jpg...
b2.jpg...
e6.jpg...
c4.jpg...
g9.jpg...
h3.jpg...
a7.jpg...
d9.jpg...
e7.jpg...
h6.jpg...
a6.jpg...
b7.jpg...
f1.jpg...
a5.jpg...
f9.jp

In [7]:
for i in range(len(pi_m1_arr)):
  pi_m1_arr[i]=pi_m1_arr[i][:,0]
  pi_m2_arr[i]=pi_m2_arr[i][:,0]

In [8]:
def imshow(img):
    cv2.namedWindow("out", cv2.WINDOW_NORMAL)
    cv2.imshow("out",(img).astype(np.uint8))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [65]:
imshow(pi_m*127)

In [66]:
np.unique(pi_m*127, return_counts=True)

(array([  0, 127, 254], dtype=uint8), array([57582,  4386,  3568]))

In [9]:
x1_tr, x1_tst, y1_tr, y1_tst = train_test_split(bf_arr, pi_m1_arr, test_size=0.3, random_state=23)
x2_tr, x2_tst, y2_tr, y2_tst = train_test_split(bf_arr, pi_m2_arr, test_size=0.3, random_state=23)

In [10]:
x1_tr = np.concatenate((x1_tr[:]))
y1_tr = np.concatenate((y1_tr[:]))
x1_tst = np.concatenate((x1_tst[:]))
y1_tst = np.concatenate((y1_tst[:]))
x2_tr = np.concatenate((x2_tr[:]))
y2_tr = np.concatenate((y2_tr[:]))
x2_tst = np.concatenate((x2_tst[:]))
y2_tst = np.concatenate((y2_tst[:]))

In [11]:
print(x1_tr.shape)
print(y1_tr.shape)
print(x1_tst.shape)
print(y1_tst.shape)
print(x2_tr.shape)
print(y2_tr.shape)
print(x2_tst.shape)
print(y2_tst.shape)

(3014656, 1)
(3014656,)
(1376256, 1)
(1376256,)
(3014656, 1)
(3014656,)
(1376256, 1)
(1376256,)


In [12]:
lr = LogisticRegression(max_iter=1000)

lr.fit(x1_tr,y1_tr)
y1_pred = lr.predict_proba(x1_tst)[::,1]
print(quality_metrics(y1_tst, y1_pred ))

lr.fit(x2_tr,y2_tr)
y2_pred = lr.predict_proba(x2_tst)[::,1]
print(quality_metrics(y2_tst, y2_pred ))

(0.7783706075083197, 0.8780038016183036, 0.4445252657205629, 0.9526122415608042)
(0.929126760526679, 0.9569476899646577, 0.0, 1.0)


In [14]:
imshow(y1_pred[:256*256].reshape(256,256)*255)

In [15]:
def jaccard(img0, img1): return np.logical_and(img0, img1).sum()/np.logical_or(img0, img1).sum()

In [80]:
#th = []
#for thresh in np.arange(0.01,0.99,0.01):
thresh = 0.5
sim = []
for i in range(int(len(y1_tst)/(256*256))):
    lo = i*256*256
    hi = (i+1)*256*256
    pred = (y1_pred[lo:hi]>thresh).reshape(256,256)
    tst = (y1_tst[lo:hi]).reshape(256,256)
    try:
        sim.append(jaccard(pred,tst))
    except:
        sim.append(0)
    imshow(np.concatenate((tst, pred),axis = 1)*255)

print(str(thresh)+ " : "+ str(np.average(sim)))
#   th.append(np.average(sim))

# print(np.argmax(th))

0.5 : 0.0


# One image

In [18]:
bf = cv2.imread("/home/maiki/Downloads/bf.JPG", cv2.IMREAD_GRAYSCALE)
pi_m_3 = cv2.imread("/home/maiki/Downloads/pi_m_3.JPG", cv2.IMREAD_GRAYSCALE)


bf_neg = 255-bf

pi_m_3_fixed = extrapolate_kmeans(pi_m_3)

trainingData = np.float32(bf_neg.reshape((-1,1)))
# labels = (pi_m_3_fixed/255).reshape((-1,1))


mask1 = pi_m_3_fixed == 1
mask2 = pi_m_3_fixed == 2



In [27]:
labels1 = (mask1).reshape((-1,1))
labels2 = (mask2).reshape((-1,1))

trainingData = trainingData/max(trainingData)


x1_tr, x1_tst, y1_tr, y1_tst = train_test_split(trainingData, labels1, test_size=0.2, random_state=23)
x2_tr, x2_tst, y2_tr, y2_tst = train_test_split(trainingData, labels2, test_size=0.2, random_state=23)


In [21]:
np.unique(mask1*255)

array([  0, 255])

In [29]:
y2_tr = y2_tr[:,0]
y2_tst = y2_tst[:,0]

print(x2_tr.shape)
print(y2_tr.shape)
print(x2_tst.shape)
print(y2_tst.shape)


(3355443, 1)
(3355443,)
(838861, 1)
(838861,)


In [7]:
dt = DecisionTreeRegressor()

dt.fit(x_tr, y_tr)

y_pred = dt.predict(x_tst)

In [16]:
from sklearn.metrics import r2_score
print(r2_score(y_tst, y_pred))

0.013428493422678791
0.0016986248010660487
0.041214376145539904
0.9045785579856036


In [30]:
lr = LogisticRegression(max_iter=1000)

lr.fit(x1_tr,y1_tr)
y1_pred = lr.predict_proba(x1_tst)[::,1]
print(quality_metrics(y1_tst, y1_pred ))

lr.fit(x2_tr,y2_tr)
y2_pred = lr.predict_proba(x2_tst)[::,1]
print(quality_metrics(y2_tst, y2_pred ))

(0.9117480753095828, 0.9265897449041021, 0.0, 0.9973413810447654)
(0.9972711927099481, 0.9903595470524914, 0.9388594632768361, 0.9940886271271162)
